In [1]:
import sys
import os

SCRIPT_DIR = "c:/Users/Luke Yang/Documents/100_Luke/100_School/130_University_of_Toronto/2023_2024/APS360/Anime-popularity-predictor/transformer_synopsis"
sys.path.append(os.path.dirname(SCRIPT_DIR))

os.chdir('C:\\Users\\Luke Yang\\Documents\\100_Luke\\100_School\\130_University_of_Toronto\\2023_2024\\APS360\\Anime-popularity-predictor')

import torch
import torch.nn as nn
import pandas as pd
import common
from common import model
from common import evaluate
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import transformer_synopsis_model
from transformer_synopsis_model import *
from common import separate_test

c:\Users\Luke Yang\Documents\100_Luke\100_School\130_University_of_Toronto\2023_2024\APS360\Anime-popularity-predictor\transformer_synopsis


In [2]:
import importlib
separate_test.separate_csv()

In [3]:
batch_size=64
cwd = os.getcwd()
train, val, max_len = get_data_loaders(f'{cwd}/data_collection/data/balanced_animes_data_max_rank=5000_no_test.csv', batch_size)

c:\Users\Luke Yang\Documents\100_Luke\100_School\130_University_of_Toronto\2023_2024\APS360\Anime-popularity-predictor\transformer_synopsis\transformer_synopsis_model.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['popularity'] = (df['popularity'] - df['popularity'].min()) / (df['popularity'].max() - df['popularity'].min())
c:\Users\Luke Yang\Documents\100_Luke\100_School\130_University_of_Toronto\2023_2024\APS360\Anime-popularity-predictor\transformer_synopsis\transformer_synopsis_model.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['synopsis'

In [4]:
# for batch in train:
#     print(batch[0].shape)
#     print(len(batch[1]))
#     break

In [5]:
lr=0.01
num_epochs=100
input_size=max_len
hidden_size=128
num_class=1

ex = SynopsisTransformer('transformer01', batch_size, lr, num_epochs, input_size, hidden_size, num_class)

In [6]:
ex.str(100)

'training/transformer01_64_0.01_100_328_128_1_100'

In [7]:
# model.train_model(ex.cuda(), train, val)

In [8]:
batch_size_list=[32, 64, 128, 256, 512]
lr_list=[1e-1, 1e-2, 1e-3, 1e-4]
num_epochs_list=[20, 50, 100, 200]
input_size=max_len
hidden_size_list=[128, 256, 512]
num_class=1

count = 0

for batch_size in batch_size_list:
    for lr in lr_list:
        for num_epochs in num_epochs_list:
            for hidden_size in hidden_size_list:
                ex = SynopsisTransformer(f"transformer_rev01_{count}", batch_size, lr, num_epochs, input_size, hidden_size, num_class)
                model.train_model(ex.cuda(), train, val)
                count += 1

c:\Users\Luke Yang\anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 300, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Luke Yang\anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([60])) that is different to the input size (torch.Size([60, 300, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Luke Yang\anaconda3\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 300, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the s